In [2]:
import numpy as np
from splex import *
from pbsig import * 
from pbsig.linalg import * 
from pbsig.vis import figure_complex
from bokeh.plotting import figure, show
from bokeh.io import output_notebook
from bokeh.layouts import row
output_notebook(verbose=False)

Loading BokehJS ...

Generate a noisy circle. For any choice of fixed radius $\epsilon \in \mathbb{R}_+$, we can construct a rips filtration $\mathcal{R}(X; \epsilon)$. For example:


In [3]:
from pbsig.vis import *
np.random.seed(1234)
theta = np.linspace(0, 2*np.pi, 8, endpoint=False)
circle = np.c_[np.sin(theta), np.cos(theta)]

eps_radius = 1.0
R = rips_filtration(circle, 1.0, p=2)

p = figure(width=300, height=300, match_aspect=True)
p.scatter(*circle.T, color="blue", size=12)
q = figure_complex(R, circle)
show(row(p, q))

In [4]:
from pbsig.persistence import ph
max_radius = 3.0
K = rips_filtration(circle, max_radius, p=2)
dgms = ph(K, engine="dionysus")
dgm = dgms[1]
show(figure_dgm(dgm))

## Changing a different parameter

Rips-persistence diagrams act linearly when their inputs are scaled, i.e. given a Rips filtration $\mathcal{R}_{\epsilon}(X)$ and its corresponding diagram $\mathrm{dgm}(\mathcal{R}_{\epsilon}(X))$, they respect: 

$$ \alpha \cdot \mathrm{dgm}(\mathcal{R}_{\epsilon}(X)) = \mathrm{dgm}(\mathcal{R}_{\epsilon}(\alpha \cdot X))$$


In [5]:
from bokeh.models import Range1d
from pbsig.vis import bin_color
alpha_family = np.linspace(0, 2.0, 100)
vine = np.c_[alpha_family*dgm["birth"], alpha_family*dgm["death"]]

p = figure_dgm()
p.scatter(vine[:,0],vine[:,1], color=bin_color(alpha_family, "turbo"))
p.x_range = Range1d(0, max(vine[:,1])*1.05)
p.y_range = Range1d(0, max(vine[:,1])*1.05)
show(p)

In [6]:
from pbsig.betti import mu_query
R = (0.5, 1.0, 1.5, 2.0)
S = simplicial_complex(faces(SimplexTree([np.arange(8)]), 2))
Q = [mu_query(S, R=R, f=flag_weight(alpha*circle), p=1, normed=False) for alpha in alpha_family]

mult_H1 = [mu() for mu in Q]
p = figure(
  width=350, height=250, 
  title=f"Circle multiplicities for R={R}", x_axis_label="alpha (scaling factor)", y_axis_label="multiplicity"
)
p.step(alpha_family, np.array(mult_H1, dtype=int))
p.yaxis.minor_tick_line_alpha = 0

q = figure_dgm(width=250, height=250)
q.scatter(vine[:,0],vine[:,1], color=bin_color(alpha_family, "turbo"))
q.x_range = Range1d(0, max(vine[:,1])*1.05)
q.y_range = Range1d(0, max(vine[:,1])*1.05)
r_width = R[1]-R[0]
r_height = R[3]-R[2]
q.rect(R[0]+r_width/2, R[2]+r_height/2, r_width, r_height, alpha=1.0, fill_alpha=0.0)


show(row(p, q))

Equivalently, we can amortize the cost of creating so many matrices by re-using the results from previous computations.


In [7]:
from pbsig.betti import MuFamily
F = [flag_weight(alpha*circle) for alpha in alpha_family]
mu_f = MuFamily(S, family=F, p=1, form="array")
mu_f.precompute(R=R, progress=True)

[████████████████████████████████████████████████████████████] 100/100



Let's look at the constitutive terms that make up this multiplicity queries


In [8]:
p = figure(
  width=450, height=250, 
  title=f"Circle multiplicities for R={R}", 
  x_axis_label="alpha (scaling factor)", 
  y_axis_label="Constititive ranks"
)
mu_terms = mu_f(smoothing=None, terms=True).T
p.step(alpha_family, mu_terms[:,0], line_color="red")
p.step(alpha_family, -mu_terms[:,1], line_color="green")
p.step(alpha_family, -mu_terms[:,2], line_color="blue")
p.step(alpha_family, mu_terms[:,3], line_color="orange")
p.step(alpha_family, mu_f(smoothing=None), line_color="black")
p.yaxis.minor_tick_line_alpha = 0
show(p)

Note that there may be regions $\bar{\mathcal{A}} \subset \mathbb{R}$ wherein all four terms in $\mu_R(\alpha)$ are $0$. Thus, even if one equipped some differentiable structure to the rank function, the gradient calculation for $\alpha \in \bar{\mathcal{A}}$ would still be $0$. In other words, there is some interval $\mathcal{A} \subset \mathbb{R}$ wherein an theoretical optimization is feasible, and outside of this region there is no hope at optimization. The bounds of $\mathcal{A}$ are given below:

In [101]:
feasible = np.flatnonzero(np.diff((abs(mu_terms).sum(axis=1) != 0).astype(int)))
f"min alpha: {alpha_family[feasible[0]]:4f}, max alpha: {alpha_family[feasible[1]]:4f}"

'min alpha: 0.242424, max alpha: 1.414141'


Now, let's look at a continuous relaxation of both the multiplicity function and its constitituive terms.


In [9]:
## constituive terms
figures = [[] for i in range(4)]
for w, normed in [(0.0,False), (0.0,True), (0.30,False), (0.30,True)]:
  mu_f.precompute(R=R, w=w, normed=normed, biased=True, progress=False)
  mu_terms = mu_f(smoothing=None, terms=True).T
  mu_terms_nuclear = mu_f(smoothing=False, terms=True).T
  mu_terms_sgn_approx = mu_f(smoothing=sgn_approx(eps=1e-1, p=1.5), terms=True).T
  for i in range(4):
    p = figure(
      width=200, height=200, 
      title=f"w:{w}, normed:{normed}", x_axis_label="alpha (scaling factor)", y_axis_label="multiplicity",
      tools="",
    )
    p.toolbar.logo = None
    p.yaxis.minor_tick_line_alpha = 0
    p.line(alpha_family, mu_terms_nuclear[:,i], line_color = "orange", line_width=2.0)
    p.line(alpha_family, mu_terms_sgn_approx[:,i], line_color = "blue",  line_width=1.5)
    p.step(alpha_family, mu_terms[:,i], line_color = "black", line_width=1.0)
    figures[i].append(p)

show(row([column(f) for f in figures]))

**TAKEAWAY**: If $w > 0$, you need degree normalization to stabilize the spectrum. Though the nuclear norm is at times quite similar to the rank, it can also differ greatly.

The weighted combinatorial laplacian is unstable whenever $w > 0$, due to the fact that $1/\epsilon$ can produce arbitrarily large values as $\epsilon \to 0^+$. This is not a problem for the normalized laplacian, as the spectrum is always bounded in the range $[0, p+2]$.

------------------------------------------------------------------------

From now on, let's only consider the normalized weighted combinatorial laplacian. Let's look at the effect of w


In [10]:
W = [0.0, 0.30, 0.60]
fig_kwargs = dict(width=200, height=150)

## Try varying epsilon in [1e-1, 100] to see interpolation of behavior
figures = [[] for i in range(len(W))]
for i,w in enumerate(W):
  mu_f.precompute(R=R, w=w, normed=True, progress=False)
  headers = ["rank", "sgn_approx", "nuclear"]
  for j, (ef, title) in enumerate(zip([None, sgn_approx(eps=1e-1, p=1.2), False], headers)):
    mu_terms = mu_f(smoothing=ef, terms=True).T
    p = figure(**(fig_kwargs | dict(title=f"w:{w}, f:{title}")), tools="")
    p.toolbar.logo = None
    p.line(alpha_family, mu_terms[:,0], line_color="red")
    p.line(alpha_family, -mu_terms[:,1], line_color="green")
    p.line(alpha_family, -mu_terms[:,2], line_color="blue")
    p.line(alpha_family, mu_terms[:,3], line_color="orange")
    p.line(alpha_family, mu_terms.sum(axis=1), line_color="black")
    p.yaxis.minor_tick_line_alpha = 0
    figures[i].append(p)

show(column([row(f) for f in figures]))

On the positive, changing `w` smoothes the objective nicely to some extent and it also expands the interval $\mu(\alpha) \neq 0$ within the feasible set. 

On the negative, the nuclear norm causes a spurious maximizer and is $0$ in the region we're trying to maximize. Moreover, varying `w` has seemingly neglible impact on the nuclear norm.

Let's zoom in on the sgn approximation

In [11]:
mu_f.precompute(R=R, w=0.30, normed=True, biased=True, progress=False)
ef = sgn_approx(eps=1e-1, p=1.2)
mu_terms = mu_f(smoothing=ef, terms=True).T
p = figure(**(fig_kwargs | dict(title=f"w:{w}, f:sgn_approx", width=450, height=250)))
p.toolbar.logo = None
p.line(alpha_family, mu_terms[:,0], line_color="red")
p.line(alpha_family, -mu_terms[:,1], line_color="green")
p.line(alpha_family, -mu_terms[:,2], line_color="blue")
p.line(alpha_family, mu_terms[:,3], line_color="orange")
p.line(alpha_family, mu_terms.sum(axis=1), line_color="black")
p.yaxis.minor_tick_line_alpha = 0

show(p)

The cancellations of the green/orange (2nd/4rth terms) and red/blue (1st/3rd terms) render ~1/3 of the feasible region useless _before_ the maximizer; the green/red and blue/orange also ~1/3 useless _after_. 

Questions that remain: 
  1. Can the region where $\hat{\mu_R} \neq 0$ be expanded?
  2. Can the objective be further smoothed _while retaining maximizers_? 
---

A common approach for retaining critical points while smoothing is the Moreau envelope of a function $f$. It is defined as attained infimum of the proximal operator of $\mathrm{prox}_{f}$. It is defined as:

$$ \mathrm{prox}_{t\lVert \cdot \rVert_\ast}(X) = \mathrm{argmin}_{Y \in \mathbb{R^{m\times m}}} \big ( \lVert Y \rVert_\ast + \frac{1}{2t}\lVert X - Y\rVert_F^2 \big ) $$

The corresponding Moreau envelope is: 

$$ M_{t\lVert \cdot \rVert_\ast}(X) = \mathrm{min}_{Y \in \mathbb{R^{m\times m}}} \big ( \lVert Y \rVert_\ast + \frac{1}{2t}\lVert X - Y\rVert_F^2 \big )$$

or equivalently, for $f = t \lVert \cdot\rVert_\ast$:

$$ M_{tf}(X) = \big ( \lVert \mathrm{prox}_{tf}(X) \rVert_\ast + \frac{1}{2t}\lVert X - \mathrm{prox}_{tf}(X) \rVert_F^2 \big )$$

Note that the quadratic term becomes blows up as $t \to 0^+$, implying smaller values of $t$ makes the corresponding Moreau envelope closer to $\lVert \cdot \rVert_\ast$. In effect,  smaller values $t$ act like an increasingly tighter neighborhood restriction in $\mathbb{R}^{m \times m}$.

In [12]:
w = 0.30
mu_f.precompute(R=R, w=w, normed=True, progress=True) # use nuclear norm

[████████████████████████████████████████████████████████████] 100/100



In [13]:
from pbsig.linalg import *
from pbsig.utility import *

mu_terms = mu_f(smoothing=True, terms=True).T
mu_mats = [mu_query_mat(S=S, R=R, f=f, p=1, w=w, form = 'array', normed=True) 
           for f in mu_f.family]
me = lambda M, t: prox_nuclear(M, t)[1]

figures = []
for t in [1e-1, 0.5, 1.0, 10.0]:
  mu_moreau = [[] for i in range(4)]
  for M1, M2, M3, M4 in mu_mats:
    mu_moreau[0].append(me(M1, t))
    mu_moreau[1].append(-me(M2, t))
    mu_moreau[2].append(-me(M3, t))
    mu_moreau[3].append(me(M4, t))
  mu_moreau = np.array(mu_moreau).T
  p = figure(**(fig_kwargs | dict(title=f"w:{w}, f:nuclear, t:{t}", tools="")))
  p.toolbar.logo = None
  p.line(alpha_family, mu_terms.sum(axis=1), line_color="black")
  p.line(alpha_family, mu_moreau.sum(axis=1), line_color="black", line_dash="dotted", line_width=1.5)
  p.yaxis.minor_tick_line_alpha = 0
  figures.append(p)

show(row(figures))


In [22]:
## Empirically get at the proximal operator of our function 
from scipy.optimize import minimize_scalar

## Recover the proximal operator for the absolute value / sgn approx
f, fname = lambda x: np.abs(x), "abs value"

def prox_f(x: float, t: float):
  dual = lambda z, t: f(z) + 1/(2*t) * (x-z)**2
  opt = minimize_scalar(dual, bracket=(x-10.0, x+10.0), args=(t))
  if opt.success:
    return opt.x, opt.fun
  else: 
    import warnings
    warnings.warn("failed to minimize")
    return x, 0.0

x_dom = np.linspace(-10, 10, 1000)
prox_figures, moreau_figures = [], []
for t in [1e-12, 1e-1, 1.0, 5.0]:
  prox_x = [prox_f(x, t)[0] for x in x_dom]
  m_env = [prox_f(x, t)[1] for x in x_dom]
  p = figure(width=200, height=200, title=f"Prox f: {fname}, t: {t}")
  p.line(x_dom, prox_x)
  m = figure(width=200, height=200, title=f"Moreau f: {fname}, t: {t}")
  m.line(x_dom, m_env)
  prox_figures.append(p)
  moreau_figures.append(m)
show(column(row(prox_figures), row(moreau_figures)))

Even if the function $f$ is not convex lower semi-continuous, the minimizers of the (smooth) Moreau regularization can match the minimizers of the original function with the right parameters. Letting $t \to 0_+$ ensures that. Since the $\mathrm{sign}$ approximation is arbitrarily close to the rank function, why not take the Moreau envelope of that? 

In [21]:
## Empirically get at the proximal operator of our function 
from scipy.optimize import minimize_scalar

## Recover the proximal operator for the absolute value / sgn approx
f, fname = lambda x: abs(sgn_approx(eps=1.2, p=1.5)(x)), "sgn approx"
x_dom = np.linspace(-10, 10, 1000)
prox_figures, moreau_figures = [], []
for t in [1e-12, 1e-1, 1.0, 5.0]:
  prox_x = [prox_f(x, t)[0] for x in x_dom]
  m_env = [prox_f(x, t)[1] for x in x_dom]
  p = figure(width=200, height=200, title=f"Prox f: {fname}, t: {t}")
  p.line(x_dom, prox_x)
  m = figure(width=200, height=200, title=f"Moreau f: {fname}, t: {t}")
  m.line(x_dom, m_env)
  prox_figures.append(p)
  moreau_figures.append(m)
show(column(row(prox_figures), row(moreau_figures)))

In the biased case, we use an `S`-curve like function replacement that maintains the same rank for each constitive term in $\mu$ 

In [15]:
w = 0.30   # hyper-parameter for R
eps = 1e-1 # hyper-parameter for sgn
sf = sgn_approx(eps=eps, p=1.5)
mu_terms = mu_f(smoothing=sf, terms=True).T
mu_mats = [mu_query_mat(S=S, R=R, f=f, p=1, w=w, biased=True, form = 'array', normed=True) 
           for f in mu_f.family]

figures = []
for t in [1e-2, 1e-1, 1.0, 10.0]:
  mu_moreau = [[] for i in range(4)]
  for M1, M2, M3, M4 in mu_mats:
    mu_moreau[0].append(moreau_loss(M1, sf, t=t))
    mu_moreau[1].append(-moreau_loss(M2, sf, t=t))
    mu_moreau[2].append(-moreau_loss(M3, sf, t=t))
    mu_moreau[3].append(moreau_loss(M4, sf, t=t))
  mu_moreau = np.array(mu_moreau).T
  p = figure(**(fig_kwargs | dict(title=f"w:{w}, f:sgn_approx, t:{t}", tools="")))
  p.toolbar.logo = None
  p.line(alpha_family, mu_terms.sum(axis=1), line_color="black")
  p.line(alpha_family, mu_moreau.sum(axis=1), line_color="black", line_dash="dotted", line_width=1.5)
  p.yaxis.minor_tick_line_alpha = 0
  figures.append(p)

show(row(figures))

In the _unbiased_ case, we center the `S`-curve around each of the parameters $(i,j,k,l) \in R$ 

In [16]:
w = 0.30   # hyper-parameter for R
eps = 1e-1 # hyper-parameter for sgn
sf = sgn_approx(eps=eps, p=1.5)
mu_terms = mu_f(smoothing=sf, terms=True).T
mu_mats = [mu_query_mat(S=S, R=R, f=f, p=1, w=w, biased=False, form = 'array', normed=True) 
           for f in mu_f.family]

figures = []
for t in [1e-2, 1e-1, 1.0, 10.0]:
  mu_moreau = [[] for i in range(4)]
  for M1, M2, M3, M4 in mu_mats:
    mu_moreau[0].append(moreau_loss(M1, sf, t=t))
    mu_moreau[1].append(-moreau_loss(M2, sf, t=t))
    mu_moreau[2].append(-moreau_loss(M3, sf, t=t))
    mu_moreau[3].append(moreau_loss(M4, sf, t=t))
  mu_moreau = np.array(mu_moreau).T
  p = figure(**(fig_kwargs | dict(title=f"w:{w}, f:sgn_approx, t:{t}", tools="")))
  p.toolbar.logo = None
  p.line(alpha_family, mu_terms.sum(axis=1), line_color="black")
  p.line(alpha_family, mu_moreau.sum(axis=1), line_color="black", line_dash="dotted", line_width=1.5)
  p.yaxis.minor_tick_line_alpha = 0
  figures.append(p)

show(row(figures))

The symmetrized version of `w` sort of uniformly smoothes out the objective, at the cost of offsetting the minimizer. It's true that the constititive matrix components may not have the correct rank for $w > 0$, they will as $w \to 0^+$. 

This motivates the strategy: start with the symmetric setting of $w > 0$ to smooth out the objective, and slowly decrease it (via temperature/cooling parameter?) to get closer to the rank.

---

The "unbiased" / symmetrized version can actually increase the domain where $\mu \neq 0$. 

In [20]:
feasible_rank = np.flatnonzero(np.diff((abs(mu_terms).sum(axis=1) != 0).astype(int)))
feasible_moreau = np.flatnonzero(np.diff((abs(mu_moreau).sum(axis=1) != 0).astype(int)))
print(f"Rank  : min alpha: {alpha_family[feasible_rank[0]]:4f}, max alpha: {alpha_family[feasible_rank[1]]:4f}")
print(f"Moreau: min alpha: {alpha_family[feasible_moreau[0]]:4f}, max alpha: {alpha_family[feasible_moreau[1]]:4f}")

Rank  : min alpha: 0.242424, max alpha: 1.414141
Moreau: min alpha: 0.161616, max alpha: 1.515152


In [ ]:
# prox_eps = lambda x,t,eps: 0.5*(-t*x + np.sqrt(t**2 * x**2 + 4 * t * eps * (x + eps))) if x >= 0 else 0.5*(-t*x - np.sqrt(t**2 * x**2 + 4 * t * eps * (x - eps)))

# wut = [np.sign(x)*prox_eps(np.abs(x), 0.5, 0.5) for x in np.linspace(-10,10,1000)]
# p = figure()
# p.line(np.linspace(-10,10,1000), wut)
# show(p)

In [ ]:
## Testing the moreau envelope 
from pbsig.linalg import prox_nuclear
X = np.random.uniform(size=(10,10))
X = X @ X.T

from pbsig.linalg import soft_threshold
t = 0.15
ew, ev = np.linalg.eigh(X)
assert np.isclose(sum(np.linalg.eigvalsh(ev @ np.diag(ew) @ ev.T)), sum(ew))
assert np.isclose(sum(np.linalg.eigvalsh(ev @ np.diag(soft_threshold(ew, 0.15)) @ ev.T)), sum(soft_threshold(ew, 0.15)))
P = ev @ np.diag(soft_threshold(ew, 0.15)) @ ev.T  #  proximal operator 
assert np.isclose(sum(np.linalg.eigvalsh(P)), sum(soft_threshold(ew, 0.15)))
me = sum(soft_threshold(ew, 0.15)) + (1/(2*0.15))*np.linalg.norm(X - P, 'fro')**2
P, mf, _ = prox_nuclear(X, t=t) # 35.36999221118293
assert np.isclose(me, mf)


A = np.random.uniform(size=(10,10))
A = A @ A.T
print(np.linalg.norm(A - X, 'fro')**2)
ew_x, U =np.linalg.eigh(X)
ew_v, V =np.linalg.eigh(A)
S = np.diag(ew_x)
D = np.diag(ew_v)

np.trace(S**2) + np.trace(D**2) - 2*np.trace(V.T @ U @ S @ U.T @ V @ D)
np.trace(S**2) + np.trace(D**2) - 2*np.trace(S**2 @ U.T @ X @ D @ X.T @ U)

sum((np.diag(A) - np.diag(X))**2)
sum((np.diag(A)**2 - np.diag(X)**2))



from pbsig.linalg import moreau
sum(moreau(ew, t))

In [ ]:
from pbsig.linalg import eigh_solver, eigvalsh_solver
from pbsig.betti import mu_query_mat
sf = sgn_approx(eps=1e-2, p=1.2)  
LM = mu_query_mat(S, R=R, f=F[jj], p=1, form="array")
Y = LM[3].todense()
#ew, ev = eigh_solver(Y)(Y)

y_shape = Y.shape
y = np.ravel(Y)
def moreau_cost(y_hat: ArrayLike, t: float = 1.0):
  Y_hat = y_hat.reshape(y_shape)
  ew = np.maximum(np.real(np.linalg.eigvals(Y_hat)), 0.0) # eiegnvalues can be negative, so we project onto the PSD cone!
  ew_yhat = sum(sf(ew)) 
  if t == 0.0: 
    return ew_yhat
  return ew_yhat + (1/(t*2))*np.linalg.norm(Y_hat - Y, "fro")**2
# ev @ np.diag(sf(ew)) @ ev.T

from scipy.optimize import minimize
y_noise = y+np.random.uniform(size=len(y), low=0, high=0.01)
w = minimize(moreau_cost, x0=y_noise, args=(0.01))
Z = w.x.reshape(y_shape)
print(f"Status: {w.status}, total error: {np.linalg.norm(Z - Y, 'fro')}, Num it: {w.nit}, Num evals: {w.nfev} \nMessage: {w.message}")

## Try a vector based optimization
from scipy.optimize import minimize
y_ew = np.linalg.eigvalsh(Y)
def sgn_approx_cost(ew_hat: ArrayLike, t: float = 1.0):
  return sum(sf(ew_hat)) + (1/(t*2)) * np.linalg.norm(sf(ew_hat) - sf(y_ew))**2
y_ew_noise = y_ew + np.random.uniform(size=len(y_ew), low=0.0, high=0.50)
w = minimize(sgn_approx_cost, x0=y_ew_noise, args=(0.5), tol=1e-15, method="Powell")
print(f"Status: {w.status}, total error: {np.linalg.norm(y_ew - w.x)}, Num it: {w.nit}, Num evals: {w.nfev} \nMessage: {w.message}")



eigvalsh_solver(Z)(Z)

mu_query(S, R, f)
solver = eigh_solver(x)
ew, ev = solver(x)


x0 

# j = np.searchsorted(alpha_thresholds, 0.90)
# mu = mu_query(S, R=np.append(R, 0.35), f=F[j], p=1, normed=True)
# mu(smoothing=None, terms=False) # 1 
# mu(smoothing=None, terms=True) # 8,  -8, -16,  17
# mu(smoothing=sgn_approx(eps=1e-2, p=1.2), terms=False) # 1.0004954387928944
# mu(smoothing=sgn_approx(eps=0.90, p=1.0), terms=False) # 0.5891569860030863
# mu(smoothing=False, terms=True) #  8., -16., -16.,  24.

# jj = np.searchsorted(alpha_thresholds, 1.15)
# f=F[jj]
# spectral_rank(EW[0])

# mu = mu_query(S, R=np.append(R, 0.35), f=F[jj], p=1, normed=True)
# mu(smoothing=False, terms=True)

In [ ]:
mu_f.precompute(R=R, normed=False, progress=True)

Use discrete vineyards to get an idea of what the

# st = SimplexTree(complete_graph(X.shape\[0\]))

# st.expand(2)

# S = st

N, M = 20, 24 SW = sliding_window(sw_f, bounds=(0, 12*np.pi)) d, tau = sw_parameters(bounds=(0,12*np.pi), d=M, L=6) #S = delaunay_complex(F(n=N, d=M, tau=tau)) X = SW(n=N, d=M, tau=tau) \# r = enclosing_radius(X)\*0.60 \# S = rips_complex(X, r, 2) show(plot_complex(S, X\[:,:2\]))

## Plot

scatters = \[\] for t in np.linspace(0.50*tau, 1.50*tau, 10): X_delay = SW(n=N, d=M, tau=t) p = figure(width=150, height=150, toolbar_location=None) p.scatter(*pca(X_delay).T) scatters.append(plot_complex(S, pos=pca(X_delay), width=125, height=125)) show(row(*scatters))

from pbsig.persistence import ph from pbsig.vis import plot_dgm K = filtration(S, f=flag_weight(X)) dgm = ph(K, engine="dionysus") plot_dgm(dgm\[1\])

from pbsig.betti import MuSignature, mu_query from pbsig.linalg import \* R = np.array(\[4, 4.5, 6.5, 7.5\]) T_dom = np.append(np.linspace(0.87*tau, tau, 150, endpoint=False), np.linspace(tau, tau*1.12, 150, endpoint=False)) t_family = \[flag_weight(SW(n=N, d=M, tau=t)) for t in T_dom\]

MU_f = mu_query(S, R=R, f=flag_weight(SW(n=N, d=M, tau=tau)), p=1, form="array")

Generate a noisy circle


In [ ]:
np.random.seed(1234)
theta = np.linspace(0, 2*np.pi, 80, endpoint=False)
circle = np.c_[np.sin(theta), np.cos(theta)]
noise_scale = np.random.uniform(size=circle.shape[0], low=0.90, high=1.10)
noise_scale = np.c_[noise_scale, noise_scale]
noise = np.random.uniform(size=(10, 2), low=-1, high=1)
X = np.vstack((circle*noise_scale, noise))

## Plot the circle + noise 
p = figure(width=400, height=200)
p.scatter(X[:,0], X[:,1], color="blue")
p.scatter(*noise.T, color="red")
show(p)

````{=html}
<!-- 

In [ ]:
import line_profiler

profile = line_profiler.LineProfiler()
profile.add_function(mu_f.precompute)
profile.enable_by_count()
mu_f.precompute(R=R, w=w, normed=True, progress=True) 
profile.print_stats(output_unit=1e-3)

alpha_thresholds = np.linspace(1e-12, max_scale*r, 100)
vine = np.vstack([ph(F(alpha)[1], engine="dionysus")[1] for alpha in alpha_thresholds])

from bokeh.models import Range1d
p = figure_dgm(vine[-1,:])
p.scatter(np.ravel(vine['birth']), np.ravel(vine['death']))
p.x_range = Range1d(0, max_scale*r)
p.y_range = Range1d(0, max_scale*r)
show(p)
``` -->
````